In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.2 MB/s eta 0:00:00


In [98]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW, get_cosine_schedule_with_warmup
import pandas as pd
import numpy as np
import json
import math
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F

In [99]:
model_name = r"bert-base-uncased"
qasper_classification_path = r"/content/drive/MyDrive/Colab Notebooks/Qasper_classification.json"

device = r"cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name)
attributes = [
                "unanswerable",
                "extractive_spans",
                "yes_no",
                "abstractive"
            ]
max_token_length = 512
bert_model = AutoModel.from_pretrained(model_name, return_dict = True)
batch_size = 8

n_labels = 4
lr = 1.5e-6
warmup = 0.2
weight_decay = 0.001
n_epochs = 1
MODEL_PATH = r"/content/drive/MyDrive/Colab Notebooks/model.bin"

Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 177, in close
    self._close()
  File "/usr/lib/python3.10/multiprocessing/connection.py", line 361, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 271, in _feed
    queue_sem.release()
ValueError: semaphore or lock released too many times
Exception in thread QueueFeederThread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 239

In [100]:
class Qasper_Dataset(Dataset):

    def __init__(self, data_path, data_type, tokenizer, attributes, max_token_length = 128, sample = None):
        self.data_path = data_path
        self.data_type = data_type
        self.tokenizer = tokenizer
        self.attributes = attributes
        self.max_token_length = max_token_length
        self.data = self.__load_data__()

    def __load_data__(self):
        data_fd = open(self.data_path)
        self.data = json.load(data_fd)
        return (
            self.data['data'][self.data_type]
        )

    def __len__(self):
        return (
            len(self.data)
        )

    def __getitem__(self, index):
        context = ""
        for i in range(len(self.data[index][0])):
            if i != 0:
                context = context + " "
            context = context + self.data[index][0][i]
        # context = self.data[index][0]
        question = self.data[index][1]
        labels = [self.data[index][2][self.attributes[0]], self.data[index][2][self.attributes[1]], self.data[index][2][self.attributes[2]], self.data[index][2][self.attributes[3]]]
        labels = torch.FloatTensor(labels)
        # labels = Variable(labels, requires_grad = True)
        tokens = self.tokenizer.encode_plus(
                                            question,
                                            context,
                                            add_special_tokens=True,
                                            return_tensors='pt',
                                            truncation=True,
                                            padding='max_length',
                                            max_length=self.max_token_length,
                                            return_attention_mask = True
                                            )
        return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(), 'labels': labels}

In [101]:
train_data = Qasper_Dataset(
                data_path = qasper_classification_path,
                data_type = "train_data",
                tokenizer = tokenizer,
                attributes = attributes,
                max_token_length = 512,
                sample = None
                )

val_data = Qasper_Dataset(
                    data_path = qasper_classification_path,
                    data_type = "validation_data",
                    tokenizer = tokenizer,
                    attributes = attributes,
                    max_token_length = 512,
                    sample = None
                    )

train_dataloader = DataLoader(
                  train_data,
                  batch_size = batch_size,
                  num_workers = 2,
                  shuffle = True
              )

val_dataloader = DataLoader(
                  val_data,
                  batch_size = batch_size,
                  num_workers = 2,
                  shuffle = True
              )

In [102]:
class Qasper_Classifier(nn.Module):

    def __init__(self, model, n_labels):

        super(Qasper_Classifier, self).__init__()
        self.pretrained_model = model # bert model
        self.n_labels = n_labels

        self.hidden = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        print(f"hidden - {self.hidden.weight.requires_grad}")
        self.classifier = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.n_labels)
        print(f"classifier - {self.classifier.weight.requires_grad}")
        torch.nn.init.xavier_uniform_(self.classifier.weight)
        # print(f"loss - {self.loss_func.weight.requires_grad}")
        self.dropout = nn.Dropout(0.3)
        # print(f"dropout - {self.dropout.weight.requires_grad}")


    def forward(self, input_ids, attention_mask, labels = None):
        # print("Forward Propagation started")
        # input_ids.requires_grad = True; attention_mask,requires_grad = True
        print(f"input - {input_ids.requires_grad}, attention_mask - {attention_mask.requires_grad}")
        output = self.pretrained_model(
                                    input_ids = input_ids,
                                    attention_mask = attention_mask
                                       )
        pooled_output = torch.mean(output.last_hidden_state, 1)
        pooled_output = self.dropout(pooled_output)
        pooled_output = self.hidden(pooled_output)
        pooled_output = F.relu(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        # loss = 0
        # loss.requires_grad = True
        # logits.requires_grad = True
        # labels.required_grad = True
        # loss = Variable(loss, requires_grad = True)
        return logits

In [103]:
def train_fn(data_loader, model, optimizer, device, scheduler):

  model.train()

  for idx, data in tqdm(enumerate(data_loader), total = len(data_loader)):
    # print(data.keys())
    print(f"Training loop {idx}")
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    labels = data['labels']

    input_ids = input_ids.to(device, dtype=torch.long)
    attention_mask = attention_mask.to(device, dtype=torch.long)
    labels = labels.to(device, dtype=torch.float)

    optimizer.zero_grad()
    print(input_ids)
    print(type(input_ids))
    print(attention_mask)
    print(type(attention_mask))
    logits = model(
                  input_ids = input_ids,
                  attention_mask = attention_mask,
              )

    loss = torch.zeros(1, requires_grad=True)
    # if labels is not None:
    print(f"type - {type(logits)} type - {type(labels)}")
    loss_fn = nn.BCEWithLogitsLoss(reduction='mean')
    loss = loss_fn(logits.view(-1, len(attributes)), labels.view(-1, len(attributes)))
    print(f"loss - {loss}, logits - {logits}")
    print(f"loss__ - {loss.requires_grad}, logits__ - {logits.requires_grad}")

    loss.backward()
    optimizer.step()
    scheduler.step()

In [104]:
def eval_fn(data_loader, model):

  model.eval()

  tot_logits = []
  tot_labels = []

  with torch.no_grad():
    for idx, data in tqdm(enumerate(data_loader), total = len(data_loader)):
      print(f"Validation Loop {idx}")
      input_ids = data['input_ids']
      attention_mask = data['attention_mask']
      labels = data['labels']

      input_ids = input_ids.to(device, dtype=torch.long)
      attention_mask = attention_mask.to(device, dtype=torch.long)
      labels = labels.to(device, dtype=torch.float)

      logits = model(
                    input_ids = input_ids,
                    attention_mask = attention_mask,
                )
      logits = logits.cpu().detach().numpy().tolist()
      labels = labels.cpu().detach().numpy().tolist()

      tot_logits.extend(logits)
      tot_labels.extend(labels)

    return (
      tot_logits,
      tot_labels
   )

In [105]:
model = Qasper_Classifier(bert_model, len(attributes))
param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
  {
      "params": [
          p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
      ],
      "weight_decay": 0.001,
  },
  {
      "params": [
          p for n, p in param_optimizer if any(nd in n for nd in no_decay)
      ],
      "weight_decay": 0.0,
  },
]

num_train_steps = int(len(train_data) / batch_size * n_epochs)
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

hidden - True
classifier - True


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [106]:
def clipping_fn(logits, max_val, min_val):

  for i in range(len(logits)):
    for j in range(len(logits[i])):

        if logits[i][j] >= (max_val + min_val)/2:
          logits[i][j] = max_val
        else:
          logits[i][j] = min_val

  return (
      logits
  )

def metric_accuracy(logits, labels):
  confusion = [
      {
         'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0,
      },
      {
         'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0,
      },
      {
         'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0,
      },
      {
         'tp': 0, 'tn': 0, 'fp': 0, 'fn': 0,
      },
  ]

  for i in range(len(logits)):
    for j in range(len(logits[i])):

      if logits[i][j] == 1 and labels[i][j] == 1:
          confusion[j]['tp'] += 1

      if logits[i][j] == 0 and labels[i][j] == 0:
          confusion[j]['tn'] += 1

      if logits[i][j] == 1 and labels[i][j] == 0:
          confusion[j]['fn'] += 1

      if logits[i][j] == 0 and labels[i][j] == 1:
          confusion[j]['fp'] += 1

  print(confusion)

  accuracy = list()

  for i in range(len(confusion)):
    accuracy.append((confusion[i]['tp'] + confusion[i]['tn'])/(confusion[i]['tp'] + confusion[i]['tn'] + confusion[i]['fp'] + confusion[i]['fn']))

  return (
      accuracy
  )


In [107]:
def __run__():
    best_accuracy = 0
    model.to(device)
    print(model)

    for epoch in range(n_epochs):
        train_fn(train_dataloader, model, optimizer, device, scheduler)
        logits, labels = eval_fn(val_dataloader, model)
        logits = clipping_fn(logits, 1.0, 0.0)
        accuracy = metric_accuracy(logits, labels)
        overall_accuracy = (accuracy[0] + accuracy[1] + accuracy[2] + accuracy[3])/len(accuracy)

        print(f"Detailed accuracy after {epoch} epoch:")
        print(f"unanswerable accuarcy: {accuracy[0]}")
        print(f"extractive accuarcy: {accuracy[1]}")
        print(f"yes_no accuarcy: {accuracy[2]}")
        print(f"abstractive accuarcy: {accuracy[3]}")
        print(f"Overall accuarcy: {overall_accuracy}")
        print(f"Best accuarcy: {best_accuracy}")

        if overall_accuracy > best_accuracy:
            torch.save(model.state_dict(), MODEL_PATH)
            best_accuracy = overall_accuracy
            print(best_accuracy)
            print("Model Updated")

In [108]:
__run__()

Qasper_Classifier(
  (pretrained_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

  0%|          | 0/289 [00:00<?, ?it/s]

Training loop 0
tensor([[  101,  2129,  2079,  ...,     0,     0,     0],
        [  101,  2129,  2064,  ...,     0,     0,     0],
        [  101,  2054,  2024,  ...,     0,     0,     0],
        ...,
        [  101,  2054, 15756,  ...,     0,     0,     0],
        [  101,  2029,  2944,  ...,     0,     0,     0],
        [  101,  2054,  2003,  ...,     0,     0,     0]], device='cuda:0')
<class 'torch.Tensor'>
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
<class 'torch.Tensor'>
input - False, attention_mask - False
type - <class 'torch.Tensor'> type - <class 'torch.Tensor'>
loss - 0.6920968890190125, logits - tensor([[ 0.1180, -0.3500, -0.4331,  0.1804],
        [-0.0655, -0.2152, -0.0058,  0.1718],
        [-0.2948, -0.1481, -0.1785, -0.0338],
        [ 0.0591, -0.2294, -0.0189, -0.0320],
      

  0%|          | 1/289 [00:00<04:05,  1.18it/s]

Training loop 1
tensor([[ 101, 2054, 4895,  ...,    0,    0,    0],
        [ 101, 2054, 2951,  ...,    0,    0,    0],
        [ 101, 2079, 2027,  ...,    0,    0,    0],
        ...,
        [ 101, 2129, 2515,  ..., 1035, 1045,  102],
        [ 101, 2106, 2027,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0]], device='cuda:0')
<class 'torch.Tensor'>
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
<class 'torch.Tensor'>
input - False, attention_mask - False
type - <class 'torch.Tensor'> type - <class 'torch.Tensor'>
loss - 0.6461156606674194, logits - tensor([[-0.3202, -0.2056, -0.3761, -0.1771],
        [-0.4019,  0.0182, -0.2976, -0.1176],
        [ 0.0619, -0.0971, -0.5334, -0.2860],
        [-0.3154,  0.0052, -0.3687,  0.1308],
        [-0.1562, -0.0896, -0.5025, -0.296

...

Validation Loop 192
input - False, attention_mask - False


 99%|█████████▉| 193/194 [00:48<00:00,  4.02it/s]

Validation Loop 193
input - False, attention_mask - False


100%|██████████| 194/194 [00:48<00:00,  3.98it/s]


[{'tp': 0, 'tn': 1552, 'fp': 0, 'fn': 0}, {'tp': 927, 'tn': 331, 'fp': 34, 'fn': 260}, {'tp': 158, 'tn': 1367, 'fp': 2, 'fn': 25}, {'tp': 153, 'tn': 1098, 'fp': 278, 'fn': 23}]
Detailed accuracy after 1 epoch:
unanswerable accuarcy: 1.0
extractive accuarcy: 0.8105670103092784
yes_no accuarcy: 0.9826030927835051
abstractive accuarcy: 0.8060567010309279
Overall accuarcy: 0.8998067010309279
Best accuarcy: 0.8990012886597938
0.8998067010309279
Model Updated


In [185]:
context = ["Alice and Bob are playing chess. Alice is a beginner and Bob is an expert. Alice makes a blunder and loses her queen. Bob decides to give her a chance and offers to trade his queen for a pawn. Alice accepts the offer"]
question1 = "Was Alice playing Chess?"
question2 = "Why did Bob offer to trade his queen to Alice for a pawn?"
question3 = "Comment down on the expertise level of the players."

dummy_init = {
                    "unanswerable": False,
                    "extractive_spans": False,
                    "yes_no": False,
                    "abstractive": False
                }

layer1 = [context, question1, dummy_init]
layer2 = [context, question2, dummy_init]
layer3 = [context, question3, dummy_init]
# json.dump(data, f, indent=4)

predictData = json.load(open(r"/content/drive/MyDrive/Colab Notebooks/examplepredict.json"))
# predictData['data']['predict_data'][0][0] = context
# predictData['data']['predict_data'][0][1] = question

predictData['data']['predict_data'] = [layer1, layer2, layer3]

print(predictData['data']['predict_data'][0][1])

with open("/content/drive/MyDrive/Colab Notebooks/examplepredict.json", "w") as f:
  json.dump(predictData, f, indent=4)

Was Alice playing Chess?


In [186]:
# device = "cpu"
predict_data = Qasper_Dataset(
                    data_path = r"/content/drive/MyDrive/Colab Notebooks/examplepredict.json",
                    data_type = "predict_data",
                    tokenizer = tokenizer,
                    attributes = attributes,
                    max_token_length = 512,
                    sample = None
                    )

predict_dataloader = DataLoader(
                  predict_data,
                  batch_size = 3,
                  num_workers = 1,
                  shuffle = False
              )

In [187]:
for idx, data in tqdm(enumerate(predict_dataloader), total = len(predict_dataloader)):
    # print(data.keys())
    # device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    # print(f"Training loop {idx}")
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    # labels = data['labels']

    input_ids = input_ids.to(device, dtype=torch.long)
    attention_mask = attention_mask.to(device, dtype=torch.long)
    # labels = labels.to(device, dtype=torch.float)

    optimizer.zero_grad()
    # print(input_ids)
    # print(type(input_ids))
    # print(attention_mask)
    # print(type(attention_mask))
    logits = model(
                  input_ids = input_ids,
                  attention_mask = attention_mask,
              )
    sigmoid = nn.Sigmoid()
    # logits = clipping_fn(logits, 1.0, 0.0)
    logits = sigmoid(logits)
    print(logits)

100%|██████████| 1/1 [00:00<00:00,  5.29it/s]

Training loop 0
input - False, attention_mask - False
tensor([[0.0058, 0.0159, 0.9633, 0.0451],
        [0.0027, 0.7935, 0.0072, 0.1872],
        [0.0024, 0.6740, 0.0208, 0.1372]], device='cuda:0',
       grad_fn=<SigmoidBackward0>)


In [ ]:
context = "Hello my name name is Preetam."
question = "What is my name?"

print("Loading Model...")

classification_model = Qasper_Classifier(bert_model, len(attributes))
classification_model.load_state_dict(torch.load(MODEL_PATH))

# classification_model = torch.load(MODEL_PATH)
print(classification_model)

In [ ]:
print(input_ids)
print(type(input_ids))
print(attention_mask)
print(type(attention_mask))

In [ ]:
# print(classification_model)

In [ ]:
# optimizer.zero_grad()
# print(model)
# logits = model(
#               input_ids = input_ids,
#               attention_mask = attention_mask,
#           )

In [ ]:
print(logits)